In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tqdm
import os 

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = 'MIG-4c6deb33-4c3f-5990-89bf-891bd00dac17'

In [3]:
training_file = "balanced_training_data.txt"
validation_file = "balanced_validation_data.txt"
test_file = "balanced_testing_data.txt"


In [ ]:
def extract_unique_labels(data_file):
    """extracts unique labels from the dataset

    Args:
        data_file (str): File path which contains the images. Each line is the image's path.

    Returns:
        list: A list which is sorted with the unique labels found in the dataset.
    """
    unique_labels = []
    with open(data_file, 'r') as f:
        for line in f:
            img_path = line.strip()
            label = img_path.split('/')[-2]  
            
            if label not in unique_labels:
                unique_labels.append(label)
                
    return sorted(unique_labels) 

In [5]:
unique_labels = extract_unique_labels(training_file)
label_to_index = {label: idx for idx, label in enumerate(unique_labels)}
num_classes = len(unique_labels)


In [6]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,)) 
])

In [ ]:
class CustomTransportDataset(Dataset):
    """ 
    Args:
        Dataset (str): Loads the images and their labels. 
    """
    def __init__(self, data_file, label_to_index, transform=None):
        """Initializer of the class with the data files, labels and transform choice 

        Args:
            data_file (str): image's path 
            label_to_index (dict): Dictionary with the mapping labels and indices.
            transform (callable, optional): Optional transformations to each image. 
        """
        self.file_paths = []
        self.labels = []
        self.transform = transform
        self.label_to_index = label_to_index

        valid_extensions = ('.png') 
        with open(data_file, 'r') as f:
            for line in f:
                img_path = line.strip()
                if img_path.lower().endswith(valid_extensions): 
                    self.file_paths.append(img_path)
                    label = img_path.split('/')[-2]  
                    if label in self.label_to_index:
                        self.labels.append(self.label_to_index[label])
                    else:
                        print(f"Label {label} not found in label_to_index mapping.")

    def __len__(self):
        """Total number of image paths

        Returns:
            int: Number of images in the dataset
        """
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Gets the image from the index, applies any available transformations

        Args:
            idx (int): Index of the image

        Returns:
            tuple: index of the image with the label
        """
        img_path = self.file_paths[idx]
        label = self.labels[idx]
        img = Image.open(img_path).convert('L')  

        

        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)

        return img, label


In [ ]:
class TransportClass(nn.Module):
    """A CNN for vehicle image classification, consists of 4 convolutional layers
        each one followed by a MaxPool layer. 
    Args:
        num_classes(int): The number of unique classes which defines the size of the last output layer
    """
    def __init__(self, num_classes):
        """Initializer of the TransportClass with the number of output classes

        Args:
            num_classes (int): The number of unique classes which defines the size of the last output layer
        """
        super(TransportClass, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),  
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, num_classes),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        """Forward pass of the CNN model 

        Args:
            x (torch.Tensor): Input tensor 

        Returns:
            torch.Tensor: Output tensor
        """
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  
        x = self.fc_layers(x)
        return x


In [ ]:
def train_model(train_loader, model, criterion, optimizer, num_epochs=5):
    """Training of the CNN model using the already given model, loss function optimizer and a number of epochs

    Args:
        train_loader (Dataloader): loads batches of input images and labels
        model (nn.Module): The neural network model which is trained 
        criterion (callable): the loss function
        optimizer (Optimizer): Adjusts model weights on gradients
        num_epochs (int): Number of epochs that the model will be trained on

    Returns:
        nn.Module: The trained model 
    """
    device = torch.device("cuda:1")
    model.to(device)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        running_loss = 0
        for inputs, labels in tqdm.tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Training Loss after epoch {epoch + 1}: {running_loss / len(train_loader)}")

    return model


In [10]:
model = TransportClass(num_classes=num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

In [ ]:
def evaluate_model(loader, model):
    """Evaluates the performance of the trained model after a specified number of epochs.
    Using sklearn evaluation metrics: accuracy, precision, recall, F1 score.

    Args:
        loader (Dataloader): Batches of test or validation data 
        model (nn.Module): The already trained model in order to get the evaluation metrics.

    Returns:
        tuple: Contains the accuracy, the precision, the recall and the f1 score
    """
    model.eval() 
    device = torch.device("cuda:1")
    model.to(device)

    predicted_labels = []
    true_labels = []

    with torch.no_grad():  
        for inputs, labels in tqdm.tqdm(loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            predlab = outputs.argmax(dim=1).cpu().numpy()  
            predicted_labels.extend(predlab)
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    recall = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    return accuracy, precision, recall, f1


In [12]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop((64, 64), scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2,contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(), 
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

eval_transform = transforms.Compose([
    
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))

])

In [13]:
train_dataset = CustomTransportDataset(training_file, label_to_index, transform=train_transform)
val_dataset = CustomTransportDataset(validation_file, label_to_index, transform=eval_transform)
test_dataset = CustomTransportDataset(test_file, label_to_index, transform=eval_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [14]:
print(f"Training dataset size: {len(train_dataset)}")
print(f"Number of unique labels: {num_classes}")
print(f"Label to index mapping: {label_to_index}")

trained_model = train_model(train_loader, model, criterion, optimizer, num_epochs=120)


Training dataset size: 6592
Number of unique labels: 5
Label to index mapping: {'bus': 0, 'car': 1, 'motorcycle': 2, 'train': 3, 'truck': 4}
Epoch 1/120


100%|█████████████████████████████████████████████████████████████████████████████████| 206/206 [00:14<00:00, 14.53it/s]


Training Loss after epoch 1: 1.9097156322118147
Epoch 2/120


100%|█████████████████████████████████████████████████████████████████████████████████| 206/206 [00:14<00:00, 14.69it/s]


Training Loss after epoch 2: 1.272204956383381
Epoch 3/120


...

Training Loss after epoch 118: 0.09101892394738191
Epoch 119/120


100%|█████████████████████████████████████████████████████████████████████████████████| 206/206 [00:14<00:00, 14.41it/s]


Training Loss after epoch 119: 0.09117154552159351
Epoch 120/120


100%|█████████████████████████████████████████████████████████████████████████████████| 206/206 [00:14<00:00, 14.58it/s]

Training Loss after epoch 120: 0.08274305631628133


In [15]:
val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(val_loader, trained_model)
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation Precision: {val_precision}")
print(f"Validation Recall: {val_recall}")
print(f"Validation F1-Score: {val_f1}")

test_accuracy, test_precision, test_recall, test_f1 = evaluate_model(test_loader, trained_model)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"Test F1-Score: {test_f1}")

100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 17.70it/s]


Validation Accuracy: 0.7769230769230769
Validation Precision: 0.7880058206978517
Validation Recall: 0.776923076923077
Validation F1-Score: 0.7731693039233265


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [00:01<00:00, 18.37it/s]

Test Accuracy: 0.765934065934066
Test Precision: 0.7850923146604915
Test Recall: 0.7659340659340659
Test F1-Score: 0.7649640860922502


In [16]:
torch.save(trained_model.state_dict(), "GrayscaleUpsampledAugmentedmodelfinal.pth")
print("Model saved successfully.")

Model saved successfully.
